In [1]:
!pip install gdown

In [2]:
cd /kaggle

/kaggle


In [3]:
#image https://drive.google.com/file/d/11lE_69AXIOM8h0oKKCmXOS08AAXD3252/view?usp=drive_link
#anno https://drive.google.com/file/d/1f2j3EiOf5McgpmLW-jrWdNs2jjd5WQxN/view?usp=drive_link
!gdown 11lE_69AXIOM8h0oKKCmXOS08AAXD3252
!gdown 1f2j3EiOf5McgpmLW-jrWdNs2jjd5WQxN 

Downloading...
From (original): https://drive.google.com/uc?id=11lE_69AXIOM8h0oKKCmXOS08AAXD3252
From (redirected): https://drive.google.com/uc?id=11lE_69AXIOM8h0oKKCmXOS08AAXD3252&confirm=t&uuid=99302d1a-0048-4628-8b8c-c27511455012
To: /kaggle/Images.zip
100%|██████████████████████████████████████| 10.2G/10.2G [02:58<00:00, 57.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1f2j3EiOf5McgpmLW-jrWdNs2jjd5WQxN
To: /kaggle/labelTxt.zip
100%|███████████████████████████████████████| 2.08M/2.08M [00:00<00:00, 177MB/s]


In [4]:
!mkdir  /kaggle/working/data

!unzip Images.zip -d /kaggle/working/data

# Unzip labelTxt.zip into that folder
!unzip labelTxt.zip -d /kaggle/working/data

Archive:  Images.zip
   creating: /kaggle/working/data/Images/
  inflating: /kaggle/working/data/Images/P0000.png  
  inflating: /kaggle/working/data/Images/P0001.png  
  inflating: /kaggle/working/data/Images/P0002.png  
  inflating: /kaggle/working/data/Images/P0005.png  
  inflating: /kaggle/working/data/Images/P0008.png  
  inflating: /kaggle/working/data/Images/P0010.png  
  inflating: /kaggle/working/data/Images/P0011.png  
  inflating: /kaggle/working/data/Images/P0012.png  
  inflating: /kaggle/working/data/Images/P0013.png  
  inflating: /kaggle/working/data/Images/P0018.png  
  inflating: /kaggle/working/data/Images/P0020.png  
  inflating: /kaggle/working/data/Images/P0021.png  
  inflating: /kaggle/working/data/Images/P0022.png  
  inflating: /kaggle/working/data/Images/P0023.png  
  inflating: /kaggle/working/data/Images/P0025.png  
  inflating: /kaggle/working/data/Images/P0029.png  
  inflating: /kaggle/working/data/Images/P0030.png  
  inflating: /kaggle/working/data/Im

In [7]:
cd /kaggle/working

/kaggle/working


In [5]:
import os, random, math, json, gc
from pathlib import Path
from collections import defaultdict
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
import torchvision
from torchvision.transforms import functional as TF
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.ops import roi_align, nms, box_iou
from torchvision.models.detection.image_list import ImageList  # Add this import

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Torch:", torch.__version__, "TorchVision:", torchvision.__version__, "Device:", DEVICE)
print("Number of GPUs:", torch.cuda.device_count())

DOTA_ROOT = "/kaggle/working/data"
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]
IMG_MIN, IMG_MAX = 800, 1333
TARGET_SIZE = (800, 800)

def set_seed(s=42):
    random.seed(s); np.random.seed(s); torch.manual_seed(s); torch.cuda.manual_seed_all(s)
set_seed(42)

# Unchanged utility functions: resize_keep_ratio, to_tensor_norm, obb_to_hbb, DOTAIndex, EpisodeSampler
def resize_keep_ratio(img, min_side=IMG_MIN, max_side=IMG_MAX, target_size=TARGET_SIZE):
    w, h = img.size
    scale = min(min_side / min(h, w), max_side / max(h, w))
    new_w, new_h = int(w * scale), int(h * scale)
    img_r = img.resize((new_w, new_h), Image.BILINEAR)
    pad_w = target_size[1] - new_w
    pad_h = target_size[0] - new_h
    pad_left = pad_w // 2
    pad_top = pad_h // 2
    img_padded = Image.new("RGB", target_size, (0, 0, 0))
    img_padded.paste(img_r, (pad_left, pad_top))
    return img_padded, scale, (pad_left, pad_top)

def to_tensor_norm(img):
    t = TF.to_tensor(img)
    t = TF.normalize(t, mean=MEAN, std=STD)
    return t

def obb_to_hbb(pts8):
    xs = pts8[0::2]; ys = pts8[1::2]
    x1, y1, x2, y2 = min(xs), min(ys), max(xs), max(ys)
    return [x1, y1, x2, y2]

class DOTAIndex:
    def __init__(self, root):
        self.root = Path(root)
        self.img_dir = self.root / "Images"
        self.lbl_dir = self.root / "labelTxt"
        assert self.img_dir.is_dir() and self.lbl_dir.is_dir(), "DOTA folders not found."
        self.img_paths = sorted([p for p in self.img_dir.iterdir() if p.suffix.lower() in [".jpg",".png",".jpeg"]])
        self.records = []
        self._load()

    def _load(self):
        for img_path in self.img_paths:
            name = img_path.stem
            txt = self.lbl_dir / f"{name}.txt"
            boxes, labels = [], []
            if txt.exists():
                with open(txt, "r") as f:
                    for line in f:
                        parts = line.strip().split()
                        if len(parts) < 10:
                            continue
                        pts = list(map(float, parts[:8]))
                        cls = parts[8]
                        x1, y1, x2, y2 = obb_to_hbb(pts)
                        boxes.append([x1, y1, x2, y2])
                        labels.append(cls)
            if boxes:
                boxes = np.array(boxes, np.float32)
                labels = np.array(labels, dtype=object)
                classes = set(labels.tolist())
            else:
                boxes = np.zeros((0,4), np.float32)
                labels = np.zeros((0,), dtype=object)
                classes = set()
            self.records.append(dict(img_path=img_path, boxes=boxes, labels=labels, classes=classes))

    def class_list(self):
        allc = set()
        for r in self.records:
            allc |= r["classes"]
        return sorted(list(allc))

BASE_CLASSES = ['ship','plane','storage-tank','baseball-diamond','tennis-court',
                'basketball-court','ground-track-field','harbor','bridge','large-vehicle']
NOVEL_CLASSES = ['small-vehicle','helicopter','roundabout','soccer-ball-field','swimming-pool']

dota = DOTAIndex(DOTA_ROOT)
all_classes = dota.class_list()
print(f"Found {len(all_classes)} classes in split:", all_classes)
print("Using base:", BASE_CLASSES)
print("Using novel:", NOVEL_CLASSES)

class EpisodeSampler(Dataset):
    def __init__(self, dota: DOTAIndex, base_classes, n_way=3, k_shot=5, q_query=10, episodes=1000):
        self.dota = dota
        self.n_way, self.k_shot, self.q_query, self.episodes = n_way, k_shot, q_query, episodes
        self.base_classes = base_classes
        self.cls2img = defaultdict(list)
        for i, r in enumerate(dota.records):
            for c in r["classes"]:
                if c in base_classes:
                    self.cls2img[c].append(i)
        available = [c for c in base_classes if len(self.cls2img[c]) >= (k_shot+q_query)]
        if len(available) < n_way:
            print("WARNING: some base classes lack enough images; reduce K/Q or change split.")
        self.base_classes = available if len(available) >= n_way else base_classes

    def __len__(self): return self.episodes

    def _load_img_boxes(self, idx, allowed):
        rec = self.dota.records[idx]
        img = Image.open(rec["img_path"]).convert("RGB")
        img_r, scale, (pad_left, pad_top) = resize_keep_ratio(img)
        img_t = to_tensor_norm(img_r)
        boxes = rec["boxes"] * scale
        boxes[:, [0, 2]] += pad_left
        boxes[:, [1, 3]] += pad_top
        labels = rec["labels"]
        keep = [i for i, c in enumerate(labels) if c in allowed]
        boxes = boxes[keep] if len(keep) else np.zeros((0,4), np.float32)
        labels = labels[keep] if len(keep) else np.array([], dtype=object)
        return img_t, torch.tensor(boxes, dtype=torch.float32), labels

    def __getitem__(self, idx):
        epi_classes = random.sample(self.base_classes, self.n_way)
        allowed = set(epi_classes)
        remap = {c:i for i,c in enumerate(epi_classes)}
        support_imgs, support_boxes, support_labels = [], [], []
        query_imgs, query_boxes, query_labels = [], [], []
        for c in epi_classes:
            ids = random.sample(self.cls2img[c], self.k_shot + self.q_query)
            sup_ids, qry_ids = ids[:self.k_shot], ids[self.k_shot:]
            for sid in sup_ids:
                img_t, boxes, labels = self._load_img_boxes(sid, allowed)
                mask = [i for i, cc in enumerate(labels) if cc == c]
                if len(mask):
                    support_imgs.append(img_t)
                    support_boxes.append(boxes[mask])
                    support_labels.append(torch.full((len(mask),), remap[c], dtype=torch.long))
            for qid in qry_ids:
                img_t, boxes, labels = self._load_img_boxes(qid, allowed)
                if boxes.shape[0] == 0:
                    continue
                lbs = torch.tensor([remap[cc] for cc in labels], dtype=torch.long)
                query_imgs.append(img_t)
                query_boxes.append(boxes)
                query_labels.append(lbs)
        pack = lambda xs: torch.stack(xs) if len(xs) and xs[0].ndim==3 else xs
        return {
            "support_imgs": pack(support_imgs),
            "support_boxes": support_boxes,
            "support_labels": support_labels,
            "query_imgs": pack(query_imgs),
            "query_boxes": query_boxes,
            "query_labels": query_labels,
            "epi_classes": epi_classes
        }

class ProtoFRCNN(nn.Module):
    def __init__(self):
        super().__init__()
        m = fasterrcnn_resnet50_fpn(weights="DEFAULT")
        self.backbone = m.backbone
        self.rpn = m.rpn
        self.roi_pool = m.roi_heads.box_roi_pool
        self.box_head = m.roi_heads.box_head
        self.feat_dim = 1024

    def rpn_proposals(self, imgs, topk=800):
        # Wrap tensor in ImageList for RPN compatibility
        image_sizes = [(imgs.shape[-2], imgs.shape[-1])] * imgs.shape[0]  # [H, W] for each image
        image_list = ImageList(tensors=imgs, image_sizes=image_sizes)
        feats = self.backbone(imgs)
        props, _ = self.rpn(image_list, feats)
        props = [p[:topk] for p in props]
        return props, feats

    def roi_feats(self, imgs, boxes):
        feats = self.backbone(imgs)
        box_indices = []
        for i, b in enumerate(boxes):
            box_indices.append(torch.full((b.shape[0],), i, dtype=torch.long, device=b.device))
        box_indices = torch.cat(box_indices) if box_indices else torch.tensor([], dtype=torch.long, device=imgs.device)
        boxes_flat = torch.cat(boxes, dim=0) if boxes else torch.tensor([], device=imgs.device)
        roi = self.roi_pool(feats, [boxes_flat], [(imgs.shape[-2], imgs.shape[-1])] * len(boxes))
        emb = self.box_head(roi)
        emb = F.normalize(emb, dim=1)
        return emb

# Unchanged functions: build_prototypes, cosine_logits, episode_train_step, infer_on_image, evaluate_ap50, visualize
def build_prototypes(model, imgs, boxes, labels, n_way):
    feats = model.roi_feats(imgs, boxes)
    labs = torch.cat(labels, 0).to(feats.device)
    protos = []
    for c in range(n_way):
        mask = (labs == c)
        if mask.any():
            p = feats[mask].mean(0)
        else:
            p = torch.zeros(model.feat_dim, device=feats.device)
        protos.append(p)
    protos = F.normalize(torch.stack(protos, 0), dim=1)
    return protos

def cosine_logits(q_feats, protos, temp=0.1):
    q_feats = F.normalize(q_feats, dim=1)
    return (q_feats @ protos.t()) / temp

def episode_train_step(model, episode, n_way, temp=0.1):
    s_imgs = episode["support_imgs"].to(DEVICE)
    s_boxes = [b.to(DEVICE) for b in episode["support_boxes"]]
    s_labs = [l.to(DEVICE) for l in episode["support_labels"]]
    q_imgs = episode["query_imgs"].to(DEVICE)
    q_boxes = [b.to(DEVICE) for b in episode["query_boxes"]]
    q_labs = [l.to(DEVICE) for l in episode["query_labels"]]
    
    model_ref = model.module if isinstance(model, nn.DataParallel) else model
    with torch.no_grad():
        protos = build_prototypes(model_ref, s_imgs, s_boxes, s_labs, n_way)
    q_feats = model_ref.roi_feats(q_imgs, q_boxes)
    y = torch.cat(q_labs, 0).to(DEVICE)
    
    logits = cosine_logits(q_feats, protos, temp)
    loss = F.cross_entropy(logits, y)
    acc = (logits.argmax(1) == y).float().mean().item()
    return loss, acc

@torch.no_grad()
def infer_on_image(model, protos, img_t, conf=0.5, nms_iou=0.5):
    model.eval()
    img_t = img_t.unsqueeze(0).to(DEVICE)
    model_ref = model.module if isinstance(model, nn.DataParallel) else model
    proposals, _ = model_ref.rpn_proposals(img_t, topk=800)
    feats = model_ref.roi_feats(img_t, proposals)
    logits = cosine_logits(feats, protos, temp=0.1)
    probs = logits.softmax(-1)
    scores, labels = probs.max(1)
    keep_boxes, keep_scores, keep_labels = [], [], []
    for c in range(protos.size(0)):
        mask = (labels == c) & (scores > conf)
        if mask.sum() == 0:
            continue
        boxes_c = proposals[0][mask]
        scores_c = scores[mask]
        keep = nms(boxes_c, scores_c, nms_iou)
        keep_boxes.append(boxes_c[keep].cpu())
        keep_scores.append(scores_c[keep].cpu())
        keep_labels.append(torch.full((len(keep),), c, dtype=torch.long))
    if len(keep_boxes) == 0:
        return torch.empty((0,4)), torch.empty((0,)), torch.empty((0,), dtype=torch.long)
    return torch.cat(keep_boxes), torch.cat(keep_scores), torch.cat(keep_labels)

@torch.no_grad()
def evaluate_ap50(model, prototypes, dota, novel_classes, n_way, num_eval=50):
    model.eval()
    model_ref = model.module if isinstance(model, nn.DataParallel) else model
    ap_scores = []
    cand = [i for i, r in enumerate(dota.records) if any(c in novel_classes for c in r["classes"])]
    if len(cand) < num_eval:
        print(f"Only {len(cand)} candidate images found; evaluating on those.")
        num_eval = len(cand)
    eval_ids = random.sample(cand, num_eval)
    
    for idx in eval_ids:
        rec = dota.records[idx]
        img = Image.open(rec["img_path"]).convert("RGB")
        img_r, scale, (pad_left, pad_top) = resize_keep_ratio(img)
        img_t = to_tensor_norm(img_r)
        gt_boxes = torch.tensor(rec["boxes"] * scale, dtype=torch.float32)
        gt_boxes[:, [0, 2]] += pad_left
        gt_boxes[:, [1, 3]] += pad_top
        gt_labels = [novel_classes.index(c) for c in rec["labels"] if c in novel_classes]
        if not gt_labels:
            continue
        gt_labels = torch.tensor(gt_labels, dtype=torch.long)
        
        pred_boxes, pred_scores, pred_labels = infer_on_image(model_ref, prototypes, img_t, conf=0.5)
        if pred_boxes.shape[0] == 0:
            ap_scores.append(0.0)
            continue
        
        ious = box_iou(pred_boxes, gt_boxes)
        tp = torch.zeros_like(pred_scores, dtype=torch.bool)
        matched = torch.zeros(gt_boxes.shape[0], dtype=torch.bool)
        for i in range(pred_scores.shape[0]):
            if pred_labels[i] not in gt_labels:
                continue
            valid_ious = ious[i][gt_labels == pred_labels[i]]
            valid_gt_idx = (gt_labels == pred_labels[i]).nonzero(as_tuple=True)[0]
            if valid_ious.numel() == 0:
                continue
            max_iou, max_idx = valid_ious.max(dim=0)
            if max_iou >= 0.5 and not matched[valid_gt_idx[max_idx]]:
                tp[i] = True
                matched[valid_gt_idx[max_idx]] = True
        
        precisions = torch.cumsum(tp, dim=0) / (torch.arange(tp.shape[0], device=tp.device) + 1)
        recalls = torch.cumsum(tp, dim=0) / max(1, gt_labels.shape[0])
        ap = precisions[recalls > 0].mean().item() if (recalls > 0).any() else 0.0
        ap_scores.append(ap)
    
    return np.mean(ap_scores) if ap_scores else 0.0

def visualize(img_t, boxes, scores, labels, class_names, out_path):
    arr = img_t.cpu().permute(1,2,0).numpy()
    arr = (arr * np.array(STD) + np.array(MEAN)).clip(0,1)
    plt.figure(figsize=(10,8)); plt.imshow(arr)
    for b, s, l in zip(boxes, scores, labels):
        x1, y1, x2, y2 = b.tolist()
        plt.gca().add_patch(plt.Rectangle((x1,y1), x2-x1, y2-y1, fill=False, lw=2, ec="lime"))
        plt.text(x1, y1-2, f"{class_names[l]} {s:.2f}", fontsize=9,
                 bbox=dict(fc="yellow", alpha=0.5, ec="none"))
    plt.axis("off"); plt.tight_layout(); plt.savefig(out_path, dpi=150); plt.close()

N_WAY = 3
K_SHOT = 5
Q_QUERY = 5
EPISODES = 600
MAX_ITERS = 1000
LR = 2e-4
TEMP = 0.1
CONF_THRESH = 0.5

ep_ds = EpisodeSampler(dota, BASE_CLASSES, n_way=N_WAY, k_shot=K_SHOT, q_query=Q_QUERY, episodes=EPISODES)
loader = DataLoader(ep_ds, batch_size=2, shuffle=True, num_workers=2, collate_fn=lambda x: x[0])

model = ProtoFRCNN().to(DEVICE)
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = nn.DataParallel(model)
model.to(DEVICE)

opt = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=5e-5)
scheduler = StepLR(opt, step_size=250, gamma=0.5)

model.train()
for it, episode in enumerate(loader, 1):
    loss, acc = episode_train_step(model, episode, N_WAY, temp=TEMP)
    opt.zero_grad()
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), 5.0)
    opt.step()
    scheduler.step()
    if it % 20 == 0:
        print(f"[iter {it:04d}] loss={loss.item():.4f} acc={acc*100:.2f}% lr={scheduler.get_last_lr()[0]:.6f}")
    if it >= MAX_ITERS:
        break

model.eval()
cls2img_novel = defaultdict(list)
for i, r in enumerate(dota.records):
    for c in r["classes"]:
        if c in NOVEL_CLASSES:
            cls2img_novel[c].append(i)

chosen_novel = NOVEL_CLASSES[:N_WAY]
sup_imgs, sup_boxes, sup_labels = [], [], []
for cid, c in enumerate(chosen_novel):
    if len(cls2img_novel[c]) < K_SHOT:
        print(f"Novel class '{c}' has < {K_SHOT} images. Pick a different split or lower K_SHOT.")
        continue
    ids = random.sample(cls2img_novel[c], K_SHOT)
    for idx in ids:
        rec = dota.records[idx]
        img = Image.open(rec["img_path"]).convert("RGB")
        img_r, scale, (pad_left, pad_top) = resize_keep_ratio(img)
        img_t = to_tensor_norm(img_r).to(DEVICE)
        mask = [i for i, cc in enumerate(rec["labels"]) if cc == c]
        if len(mask) == 0:
            continue
        boxes = rec["boxes"][mask] * scale
        boxes[:, [0, 2]] += pad_left
        boxes[:, [1, 3]] += pad_top
        boxes = torch.tensor(boxes, dtype=torch.float32, device=DEVICE)
        sup_imgs.append(img_t)
        sup_boxes.append(boxes)
        sup_labels.append(torch.full((len(mask),), cid, dtype=torch.long, device=DEVICE))

if len(sup_imgs) == 0:
    raise RuntimeError("No novel support found. Adjust NOVEL_CLASSES or check dataset.")

sup_imgs = torch.stack(sup_imgs, 0)
with torch.no_grad():
    model_ref = model.module if isinstance(model, nn.DataParallel) else model
    prototypes = build_prototypes(model_ref, sup_imgs, sup_boxes, sup_labels, N_WAY)

ap50 = evaluate_ap50(model, prototypes, dota, chosen_novel, N_WAY)
print(f"AP50 on novel classes {chosen_novel}: {ap50*100:.2f}%")

cand = [i for i, r in enumerate(dota.records) if any(c in chosen_novel for c in r["classes"])]
if len(cand):
    idx = random.choice(cand)
    rec = dota.records[idx]
    img = Image.open(rec["img_path"]).convert("RGB")
    img_r, scale, (pad_left, pad_top) = resize_keep_ratio(img)
    img_t = to_tensor_norm(img_r)
    boxes = rec["boxes"] * scale
    boxes[:, [0, 2]] += pad_left
    boxes[:, [1, 3]] += pad_top
    model_ref = model.module if isinstance(model, nn.DataParallel) else model
    boxes, scores, labels = infer_on_image(model_ref, prototypes, img_t, conf=CONF_THRESH)
    out_dir = Path("./outputs_dota"); out_dir.mkdir(parents=True, exist_ok=True)
    out_path = out_dir / f"viz_{Path(rec['img_path']).stem}.png"
    visualize(img_t, boxes, scores, labels, chosen_novel, str(out_path))
    print("Saved visualization:", out_path)
else:
    print("No candidate novel image found for viz; adjust N_WAY or NOVEL_CLASSES.")

Torch: 2.6.0+cu124 TorchVision: 0.21.0+cu124 Device: cuda
Number of GPUs: 1
Found 15 classes in split: ['baseball-diamond', 'basketball-court', 'bridge', 'ground-track-field', 'harbor', 'helicopter', 'large-vehicle', 'plane', 'roundabout', 'ship', 'small-vehicle', 'soccer-ball-field', 'storage-tank', 'swimming-pool', 'tennis-court']
Using base: ['ship', 'plane', 'storage-tank', 'baseball-diamond', 'tennis-court', 'basketball-court', 'ground-track-field', 'harbor', 'bridge', 'large-vehicle']
Using novel: ['small-vehicle', 'helicopter', 'roundabout', 'soccer-ball-field', 'swimming-pool']


Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:00<00:00, 215MB/s] 


[iter 0020] loss=0.8915 acc=52.71% lr=0.000200
[iter 0040] loss=1.0901 acc=87.27% lr=0.000200
[iter 0060] loss=1.0217 acc=61.29% lr=0.000200
[iter 0080] loss=1.0988 acc=52.28% lr=0.000200
[iter 0100] loss=1.0944 acc=16.84% lr=0.000200
[iter 0120] loss=1.0916 acc=7.23% lr=0.000200
[iter 0140] loss=1.0986 acc=45.58% lr=0.000200
[iter 0160] loss=1.1366 acc=71.04% lr=0.000200
[iter 0180] loss=0.7864 acc=80.04% lr=0.000200
[iter 0200] loss=1.0115 acc=67.07% lr=0.000100
[iter 0220] loss=1.0832 acc=23.23% lr=0.000100
[iter 0240] loss=1.2436 acc=31.99% lr=0.000100
[iter 0260] loss=1.1497 acc=44.40% lr=0.000100
[iter 0280] loss=1.0724 acc=53.85% lr=0.000100
[iter 0300] loss=1.1104 acc=23.33% lr=0.000100
AP50 on novel classes ['small-vehicle', 'helicopter', 'roundabout']: 0.00%
Saved visualization: outputs_dota/viz_P1649.png


Images.zip  input  labelTxt.zip  lib  outputs_dota  working
